In [1]:
import onnx
import onnxruntime
import numpy as np
from onnxruntime.quantization import quantize_static, QuantType, QuantFormat, CalibrationDataReader, CalibrationMethod

In [2]:
model_fp32 = './temp/yolov7-w6-pose.onnx'
model_quant = './temp/yolov7-w6-pose_quant.onnx'
onnx_model = onnx.load(model_fp32)

In [3]:
# Check the model
try:
    onnx.checker.check_model(onnx_model)
except onnx.checker.ValidationError as e:
    print("The model is invalid: %s" % e)
else:
    print("The model is valid!")

The model is valid!


In [10]:
# DataReader
class CalibrationDataProvider(CalibrationDataReader):
    def __init__(self):
        super(CalibrationDataProvider, self).__init__()
        self.counter = 0

    def get_next(self):
        if self.counter > 2:
            return None
        else:
            self.counter += 1
            return {'images': np.random.randn(1, 3, 1280, 768).astype(np.float32)}

In [11]:
dr = CalibrationDataProvider()

In [12]:
quantized_model = quantize_static(model_fp32,
                model_quant,
                dr,
                quant_format=QuantFormat.QDQ,
                optimize_model=True,
                activation_type=QuantType.QInt8,
                weight_type=QuantType.QInt8,
                calibrate_method=CalibrationMethod.MinMax)